## Test Pipelines

---



In [9]:
from setup import dask_init, dask_shutdown
import dask.distributed as dd
import dask.dataframe as ddf

import pandas as pd
import numpy as np

import ingestion as ing
import processing as prc

import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**setup**

In [10]:
# PANDAS SETUP
pd.options.mode.use_inf_as_na = True

# DASK SETUP
client, cluster = dask_init()
cluster

LocalCluster(1024be66, 'tcp://127.0.0.1:35031', workers=4, threads=8, memory=7.66 GiB)

**Pipelines**

In [18]:
users_ddf = prc.users_pipeline(persist=False)
users_ddf.head(5)

,n_ratings,n_reviews,uid,username,joined,location,country
0,7820,465,nmann08.184925,nmann08,2008-01-07 11:00:00,"United States, Washington",United States
1,2521,2504,stjamesgate.163714,StJamesGate,2007-10-08 10:00:00,"United States, New York",United States
2,1797,1143,mdagnew.19527,mdagnew,2005-05-18 10:00:00,Northern Ireland,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,2004-11-25 11:00:00,Northern Ireland,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,2003-11-20 11:00:00,Northern Ireland,Northern Ireland


In [12]:
ratings_ddf = prc.ratings_pipeline(persist=False, users_persisted=True)
ratings_ddf.head(5)

,date,bid,uid,appearance,aroma,palate,taste,overall,rating,has_review,review
0,2015-08-20 10:00:00,142544,nmann08.184925,3.25,2.75,3.25,2.75,3.00,2.88,True,"From a bottle, pours a piss yellow color with ..."
1,2015-11-14 11:00:00,120824,nmann08.184925,4.00,3.50,3.75,3.50,3.50,3.56,False,None
2,2011-11-18 11:00:00,47581,nmann08.184925,NaN,NaN,NaN,NaN,NaN,4.50,False,None
3,2013-05-03 10:00:00,93113,nmann08.184925,NaN,NaN,NaN,NaN,NaN,3.75,False,None
4,2015-01-24 11:00:00,875,nmann08.184925,4.00,3.50,3.75,4.00,3.75,3.81,False,None


In [13]:
beerscsv_ddf = prc.beerscsv_pipeline(persist=False)
beerscsv_ddf.head(5)

,bid,name,style,ba_score,bros_score,abv
0,166064,Nashe Moskovskoe,Euro Pale Lager,NaN,NaN,4.7
1,166065,Nashe Pivovskoe,Euro Pale Lager,NaN,NaN,3.8
2,166066,Nashe Shakhterskoe,Euro Pale Lager,NaN,NaN,4.8
3,166067,Nashe Zhigulevskoe,Euro Pale Lager,NaN,NaN,4.0
4,166063,Zhivoe,Euro Pale Lager,NaN,NaN,4.5


In [14]:
beers_ddf = prc.beers_pipeline(persist=True, ratings_persisted=True, beerscsv_persisted=True)
beers_ddf.head(5)

,bid,n_ratings,avg_rating,n_reviews,review_rate,ba_score,bros_score,name,style,abv
0,5,1538,3.441203,520,0.338101,78.0,89.0,Amber,Vienna Lager,4.5
1,6,2660,3.657617,968,0.363910,82.0,90.0,Turbodog,English Brown Ale,5.6
2,7,2977,3.265694,778,0.261337,74.0,88.0,Purple Haze,Fruit / Vegetable Beer,4.2
3,8,132,3.290682,70,0.530303,77.0,89.0,Wheat,American Adjunct Lager,4.2
4,9,215,3.172698,120,0.558140,74.0,83.0,Golden,American Pale Lager,4.2


In [15]:
dask_shutdown(client)